# Setup

## Import TensorFlow and NumPy

In [56]:
# Import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10

## Configure DNN settings

Here, we specify the ResNet architecture parameters:

In [2]:
# Number of classes to infer
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Depth parameter
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

## Load dataset and preprocess

We are working with the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) here.

In [3]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


## Load trained ResNet model

In [4]:
# Load model
model_name = 'weight_quant_4b_final_3'

# Model name, depth and version
model_type = 'ResNet%dv%d_%s' % (depth, version, model_name)
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), model_type)
model_full_name = 'cifar10_%s_model' % model_type
filepath = os.path.join(save_dir, model_full_name)

# Load model checkpoint
K.clear_session()
model = load_model(filepath)

ResNet20v1_weight_quant_4b_final_3


In [5]:
# Save all model parameters to a dict
weights = {}
for olayer in model.layers:
    weights[olayer.name] = olayer.get_weights()

In [115]:
# Perform baseline inference
model.evaluate(x_test, y_test, verbose=1)

 34/313 [==>...........................] - ETA: 21s - loss: 1.9078 - accuracy: 0.5708

KeyboardInterrupt: 

In [130]:
# RRAM confusion matrices for 4 levels per cell

# # Tech A
# # relax_mat = np.identity(4)
# # Tech B
# relax_mat = np.array([[99.1, 0.9, 0., 0.],
#                       [0.2, 99.6, 0.2, 0.],
#                       [0., 1.0, 99.0, 0.],
#                       [0., 0., 0.3, 99.7]]) / 100

# # Tech C
# # relax_mat = np.array([[98.2, 1.8, 0., 0.],
# #                       [2.2, 95.2, 2.6, 0.],
# #                       [0., 5.7, 92.4, 1.9],
# #                       [0., 0.3, 4.3, 95.4]]) / 100
# print(relax_mat)
# print(np.sum(relax_mat, axis=1))

full_relax_mat = np.load('confmats/techC/100000.npy')

# gmax = 128e-6
# best_levels = [0, 7, 23, 31]
# best_levels = [0, 6, 24, 31]
# best_thresh = [0, 11.56873747e-6, 61.22965932e-6, 110.32625251e-6, gmax]
pushout = 6
gmax = 40e-6
best_levels = [0, 12-pushout, 22+pushout, 31]
best_thresh = [0, 4.40881764e-6, 20.36873747e-6, 32.09619238e-6, gmax]
best_thresh_levels = np.int32(np.round(np.array(best_thresh) / gmax * 32))
print(best_thresh_levels)

print("Full confusion matrix:")
print(full_relax_mat)

relax_mat = []
relax_mat_best = full_relax_mat[:, best_levels]
# print(relax_mat_best)

# Apply thresholds
for i in range(4):
    i_lo, i_hi = best_thresh_levels[i], best_thresh_levels[i+1]
    relax_mat.append(np.sum(relax_mat_best[i_lo:i_hi, :], axis=0))

# Normalize confusion matrix
relax_mat = np.array(relax_mat)
relax_mat = relax_mat / relax_mat.sum(axis=1)[:, np.newaxis]
lsber = np.mean([relax_mat[1,0], relax_mat[0,1], relax_mat[2,3], relax_mat[3,2]])
msber = np.mean([relax_mat[1,2], relax_mat[2,1]])

print("Reduced confusion matrix:")
print(relax_mat)
print(lsber, msber)

[ 0  4 16 26 32]
Full confusion matrix:
[[351 123  48 ...   0   0   0]
 [ 97 199 145 ...   0   0   0]
 [ 38  93 132 ...   0   0   0]
 ...
 [  0   0   0 ...  78  94  85]
 [  0   0   0 ...  37  83  78]
 [  0   0   0 ...  27  65 146]]
Reduced confusion matrix:
[[0.82914573 0.16917923 0.00167504 0.        ]
 [0.0195599  0.97066015 0.00733496 0.00244499]
 [0.         0.0310559  0.83229814 0.13664596]
 [0.         0.         0.45336788 0.54663212]]
0.1946882425154534 0.019195431973150694


In [131]:
# Create weight matrices after RRAM relaxation according to the confusion matrices (4 levels per cell)

# Whether to assign two MSBs to two different RRAM cells
INTERLEAVING = True

if INTERLEAVING:
    levels_0 = [-8, -6, 0, 2]
    levels_1 = [0, 1, 4, 5]
else:
    levels_0 = [-8, -4, 0, 4]
    levels_1 = [0, 1, 2, 3]

weights_relax = {}
for layer_name in weights:
    if 'conv2d' in layer_name or 'dense' in layer_name:
        print(layer_name)
        W, b, W_max = weights[layer_name]
        W_quant = tf.quantization.fake_quant_with_min_max_args(W, -W_max, W_max, 4, narrow_range=True).numpy()
        W_relax = np.zeros_like(W)
        W_full_range = tf.quantization.fake_quant_with_min_max_args(np.linspace(-W_max, W_max, 32), -W_max, W_max, 4, narrow_range=True).numpy()
        W_quant_level = np.unique(W_full_range)
        assert(len(W_quant_level) == 15)
        for i in range(1, 16):
            if INTERLEAVING:
                sel0 = ((i >> 2) & 0b10) | ((i >> 1) & 0b1)
                sel1 = ((i >> 1) & 0b10) | (i & 0b1)
            else:
                sel0 = ((i >> 2) & 0b10) | ((i >> 2) & 0b1)
                sel1 = (i & 0b10) | (i & 0b1)
            W_mask = (W_quant == W_quant_level[i-1])
            part0 = np.random.choice(levels_0, np.sum(W_mask), p=relax_mat[sel0])
            part1 = np.random.choice(levels_1, np.sum(W_mask), p=relax_mat[sel1])
            W_relax[W_mask] = (part0 + part1) / 7 * W_max
        weights_relax[layer_name] = W_relax


conv2d_noise
conv2d_noise_1
conv2d_noise_2
conv2d_noise_3
conv2d_noise_4
conv2d_noise_5
conv2d_noise_6
conv2d_noise_7
conv2d_noise_8
conv2d_noise_9
conv2d_noise_10
conv2d_noise_11
conv2d_noise_12
conv2d_noise_13
conv2d_noise_14
conv2d_noise_15
conv2d_noise_16
conv2d_noise_17
conv2d_noise_18
conv2d_noise_19
conv2d_noise_20
dense_noise


In [132]:
# Load relaxed weights back to the model
for layer in model.layers:
    if layer.name in weights_relax:
        print(layer.name)
        W, b, W_max = layer.get_weights()
        W_relax = weights_relax[layer.name]
        layer.set_weights([W_relax, b, W_max])

conv2d_noise
conv2d_noise_1
conv2d_noise_2
conv2d_noise_3
conv2d_noise_4
conv2d_noise_5
conv2d_noise_6
conv2d_noise_7
conv2d_noise_8
conv2d_noise_9
conv2d_noise_10
conv2d_noise_11
conv2d_noise_12
conv2d_noise_13
conv2d_noise_14
conv2d_noise_15
conv2d_noise_16
conv2d_noise_17
conv2d_noise_18
conv2d_noise_19
conv2d_noise_20
dense_noise


In [133]:
# Evaluate accuracy after relaxation
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 21s 66ms/step - loss: 2.5722 - accuracy: 0.1000


In [134]:
# FINAL RESULT
pd.DataFrame([[pushout, INTERLEAVING, accuracy, lsber, msber]], columns=["Pushout", "Interleaving", "Accuracy", "LSBER", "MSBER"])

,Pushout,Interleaving,Accuracy,LSBER,MSBER
0,6,True,0.1,0.194688,0.019195


In [ ]:
# Load the confusion matrix for 16 levels per cell
relax_mat = np.load('confmats/techA/100.npy').T
relax_mat = relax_mat / np.sum(relax_mat, axis=1).reshape([16, 1])
print(relax_mat)
print(np.sum(relax_mat, axis=1))

In [ ]:
# Create weight matrices after RRAM relaxation according to the confusion matrices (16 levels per cell)
levels = np.arange(-8, 8)
weights_relax = {}
for layer_name in weights:
    if 'conv2d' in layer_name or 'dense' in layer_name:
        print(layer_name)
        W, b, W_max = weights[layer_name]
        W_quant = tf.quantization.fake_quant_with_min_max_args(W, -W_max, W_max, 4, narrow_range=True).numpy()
        W_relax = np.zeros_like(W)
        W_full_range = tf.quantization.fake_quant_with_min_max_args(np.linspace(-W_max, W_max, 32), -W_max, W_max, 4, narrow_range=True).numpy()
        W_quant_level = np.unique(W_full_range)
        assert(len(W_quant_level) == 15)
        for i in range(1, 16):
            W_mask = (W_quant == W_quant_level[i-1])
            W_int = np.random.choice(levels, np.sum(W_mask), p=relax_mat[i])
            W_relax[W_mask] = W_int / 7 * W_max
        weights_relax[layer_name] = W_relax


In [ ]:
# Load relaxed weights back to the model
for layer in model.layers:
    if layer.name in weights_relax:
        print(layer.name)
        W, b, W_max = layer.get_weights()
        W_relax = weights_relax[layer.name]
        layer.set_weights([W_relax, b, W_max])

In [ ]:
# Evaluate accuracy after relaxation
model.evaluate(x_test, y_test, verbose=1)